<a href="https://colab.research.google.com/github/nshzk8975/GCN/blob/master/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install --verbose --no-cache-dir torch-scatter
!pip install --verbose --no-cache-dir torch-sparse
!pip install --verbose --no-cache-dir torch-cluster
!pip install --verbose --no-cache-dir torch-spline-conv (optional)
!pip install torch-geometric

Created temporary directory: /tmp/pip-ephem-wheel-cache-inb61g5s
Created temporary directory: /tmp/pip-req-tracker-elnf2dlw
Created requirements tracker '/tmp/pip-req-tracker-elnf2dlw'
Created temporary directory: /tmp/pip-install-e6c6rksi
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-elnf2dlw'
Created temporary directory: /tmp/pip-ephem-wheel-cache-h6salebh
Created temporary directory: /tmp/pip-req-tracker-avm6qwgk
Created requirements tracker '/tmp/pip-req-tracker-avm6qwgk'
Created temporary directory: /tmp/pip-install-asvvxx14
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-avm6qwgk'
Created temporary directory: /tmp/pip-ephem-wheel-cache-hhxttkw3
Created temporary directory: /tmp/pip-req-tracker-md_hk8re
Created requirements tracker '/tmp/pip-req-tracker-md_hk8re'
Created temporary directory: /tmp/pip-install-7xyu4fu7
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-md_hk8re'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: 

In [0]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [0]:
edge_index = torch.tensor([[0, 1, 2, 3],
                          [0, 0, 1, 2]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1], [2]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [40]:
print(data.keys)
print(data.num_nodes)
print(data.num_features)
print(data.contains_isolated_nodes())
print(data.contains_self_loops())
print(len(data))

['x', 'edge_index']
4
1
True
True
2


# Common benchmark **datasets**

In [0]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

In [42]:
data = dataset[0]
data

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [0]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [44]:
from torch_scatter import scatter_mean
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
  print(data)

x = scatter_mean(data.x, data.batch, dim=0)
print(x)

Batch(batch=[1070], edge_index=[2, 4236], x=[1070, 3], y=[32])
Batch(batch=[1054], edge_index=[2, 3984], x=[1054, 3], y=[32])
Batch(batch=[919], edge_index=[2, 3462], x=[919, 3], y=[32])
Batch(batch=[941], edge_index=[2, 3512], x=[941, 3], y=[32])
Batch(batch=[1035], edge_index=[2, 4024], x=[1035, 3], y=[32])
Batch(batch=[1014], edge_index=[2, 3874], x=[1014, 3], y=[32])
Batch(batch=[1180], edge_index=[2, 4382], x=[1180, 3], y=[32])
Batch(batch=[909], edge_index=[2, 3652], x=[909, 3], y=[32])
Batch(batch=[1033], edge_index=[2, 4064], x=[1033, 3], y=[32])
Batch(batch=[1107], edge_index=[2, 4200], x=[1107, 3], y=[32])
Batch(batch=[1040], edge_index=[2, 4044], x=[1040, 3], y=[32])
Batch(batch=[1005], edge_index=[2, 3876], x=[1005, 3], y=[32])
Batch(batch=[909], edge_index=[2, 3488], x=[909, 3], y=[32])
Batch(batch=[1104], edge_index=[2, 4320], x=[1104, 3], y=[32])
Batch(batch=[1139], edge_index=[2, 4338], x=[1139, 3], y=[32])
Batch(batch=[1203], edge_index=[2, 4392], x=[1203, 3], y=[32])


# **Data transforms**

In [45]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', category='Airplane',
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

print(dataset[0])

Data(edge_index=[2, 17768], pos=[2518, 3], y=[2518])


# **Learning methods on graphs**

In [0]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [0]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [49]:
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7920
